In [1]:
from os.path import join

from keras.applications import VGG16, InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical

import pandas as pd
import csv
import os
import numpy as np
import json

from matplotlib import pyplot as plt
import sys
#sys.path.append("../data_preparation/")

from batch_generator import BatchGenerator, BatchSequence

from sklearn.metrics import recall_score, precision_score, f1_score

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
images_path_test = os.path.abspath('data/test/')
y_test = np.zeros((39706,1))
batch_size = 4
#test_steps = int(len(y_test)/batch_size)+1
test_steps=1
input_num = 4#len(model.input_layers)
generator =  multiple_sequence_generator(generator_num=input_num, input_dir=images_path_test, y=y_test, batch_size=batch_size)

test = [gen for gen in generator]



### Define multi sequence generator

In [6]:
def multiple_sequence_generator(generator_num, **kwargs):
    """A generator to work with multiple inputs models
    
    We create a model with a list of multiple input layers when 
    we use :func:`keras.layers.concatenate`. However,
    :class:`batch_generator.BatchGenerator` returns a single tuple 
    with two arrays, which does not fit to a model with a multiple 
    input layers. Thus, with this generator, we create the necessary 
    input for such models.
    
    Arguments:
        generator_num {int} -- number of generators should be created
        \**kwargs -- See :class:`batch_generator.BatchGenerator`
        
    Yields:
        ([ndarray,...,ndarray], ndarray) -- in the tuple; list contains feature arrays from each generator, array out of the list contains the label set
    """
   # generators_list = [BatchSequence(**kwargs) for i in range(generator_num)]
    gen = BatchSequence(**kwargs)
    return gen
   # while True:
      #  Xy_list = [gen for gen in generators_list]
        #Xy_list = [gen.next() for gen in generators_list]
      #  for x in gen
      #  yield [Xy[0] for Xy in Xy_list], Xy_list[0][1]

In [7]:
def multiple_batch_generator(generator_num, **kwargs):
    """A generator to work with multiple inputs models
    
    We create a model with a list of multiple input layers when 
    we use :func:`keras.layers.concatenate`. However,
    :class:`batch_generator.BatchGenerator` returns a single tuple 
    with two arrays, which does not fit to a model with a multiple 
    input layers. Thus, with this generator, we create the necessary 
    input for such models.
    
    Arguments:
        generator_num {int} -- number of generators should be created
        \**kwargs -- See :class:`batch_generator.BatchGenerator`
        
    Yields:
        ([ndarray,...,ndarray], ndarray) -- in the tuple; list contains feature arrays from each generator, array out of the list contains the label set
    """
    #generators_list = [BatchGenerator(**kwargs, shuffle=False) for i in range(generator_num)]
    gen = BatchGenerator(**kwargs, shuffle=False)
    
    while True:
        nx = gen.next()
        Xy_list = [nx, nx, nx, nx]
        yield [Xy[0] for Xy in Xy_list], Xy_list[0][1]

### Load stacked model

In [3]:
model = load_model('models/stacked-700_steps.h5')

### Create test predictions

In [8]:
images_path_test = os.path.abspath('data/test/')
y_test = np.zeros((39706,1))
batch_size = 64
test_steps = int(len(y_test)/batch_size)+1
input_num = 4#len(model.input_layers)
test_generator_multi = multiple_sequence_generator(generator_num=input_num,
                                               input_dir=images_path_test,
                                               y=y_test,
                                               batch_size=batch_size)
test_generator_multi2 = multiple_sequence_generator(generator_num=input_num,
                                               input_dir=images_path_test,
                                               y=y_test,
                                               batch_size=batch_size)

In [11]:
test_generator_multi

<generator object multiple_sequence_generator at 0x7f525c847eb8>

In [6]:
test_predictions = model.predict_generator(test_generator_multi, steps=test_steps, verbose=1)
np.save('stacked_predictions_test', test_predictions)

AttributeError: 'tuple' object has no attribute 'ndim'

### Create validation predictions

In [ ]:
images_path_validation = os.path.abspath('data/validation/')
y_valid = np.zeros((9897,1))
batch_size = 64
valid_steps = int(len(y_valid)/batch_size)+1
input_num = 4

valid_generator_multi = multiple_sequence_generator(generator_num=input_num,
                                               input_dir=images_path_validation,
                                               y=y_valid,
                                               batch_size=batch_size)

In [ ]:
valid_predictions = model.predict_generator(valid_generator_multi, steps=valid_steps, verbose=1)
np.save('stacked_predictions_validation', valid_predictions)